# Pipeline Survey Scenario : génération de la base LexImpact


In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [2]:
import unittest

from leximpact_common_python_libraries.config import Configuration
from leximpact_common_python_libraries.logger import logger as log

from pipeline_survey_scenario import PipelineErfsSurveyScenario
from leximpact_prepare_data.pipeline_tax_and_benefit_system import pipeline_tbs
from leximpact_prepare_data.scenario_tools.imputation_erfs import add_cases_ir_from_erfs

DEBUG : Unable to read env from /home/cgl/leximpact/leximpact-pr/.env but it could be OK. Error : [Errno 2] No such file or directory: '/home/cgl/leximpact/leximpact-pr/.env'
DEBUG : Using decouple.AutoConfig()


iaidrdi has been updated in leximpact-survey-scenario
plus_values_prelevement_forfaitaire_unique_ir has been updated in leximpact-survey-scenario
rfr_plus_values_hors_rni has been updated in leximpact-survey-scenario
rpns_imposables has been updated in leximpact-survey-scenario
rpns_autres_revenus has been updated in leximpact-survey-scenario


In [3]:
tc = unittest.TestCase()
config = Configuration(project_folder="leximpact-prepare-data")
log.debug(config)
aggregates_path = config.get("AGREGATS_PATH")
log.debug("Le logger  leximpact_common_python_libraries.logger fonctionne !")

[leximpact-common-python-libraries DEBUG @ 15:06:53] Config loaded from /home/cgl/leximpact/leximpact-prepare-data/.env OR/AND environement variables
[leximpact-common-python-libraries DEBUG @ 15:06:53] Le logger  leximpact_common_python_libraries.logger fonctionne !


## Paramétrages de la simulation
- annee_donnees : annee des données utilisées en input. Au début de la pipeline il s'agit de l'erfs-fpr 2019

- annee_pote : millésime des aggrégats issus de POTE utilisé pour l'imputation des variables manquantes. Par défaut il s'agit de la même année que l'erfs même si on a des années de pote plus récentes. Les millésimes plus récents sont utilisés dans un deuxième temps, dans la partie vieillissement jusque l'année de simulation.

- annee_de_calcul : année pour laquelle on veut réaliser une simulation. Généralement c'est l'année en cours ou l'année suivante dans le cadre du PLF.

- rebuild_imputation : Bool. Si True réalise l'imputation des variables manquantes à partir des copules de POTE. L'imputation part des tables de la collection `openfisca_erfs_fpr` (qui sont les tables construites dans openfisca_france_data) et enregistre des tables de la collection `leximpact`.

- duplicates_rows : Bool. N'est utilisé que si rebuild_imputation == True. Duplique les lignes des données, et divise les poids en conséquences, afin d'avoir plus d'observation pour l'imputation. Cela explique que les tables de la collection `leximpact` ont potentiellement plus de lignes que celles de la collection `openfisca_erfs_fpr`. La somme des poids reste la même.

- collection : Si `openfisca_erfs_fpr` : prend la sortie de openfisca_france_data. Si `leximpact` prend la version apres imputation. Si cette collection n'existe pas il faut prendre `openfisca_erfs_fpr` est rebuild_imputation = True pour la recréer.

In [4]:
# Paramètres de la simulation
annee_donnees = 2019
annee_pote = "2021"
annee_de_calcul = 2021
rebuild_imputation = True
collection = "leximpact"
survey_name = f"{collection}_{annee_donnees}"

## Chargement des données

En instanciant le `LeximpactErfsSurveyScenario()`, on appelle notre class custom elle-même basée sur `ErfsFprSurveyScenario()` (dans france-data), elle-même basée sur `AbstractErfsSurveyScenario()` (dans france-data).


Dans `ErfsFprSurveyScenario()`, on peut jouer sur:
- la liste des variables de l'ERFS-FPR que l'on ajoute dans notre table
- les valeurs par défaut de ces variables

Dans `AbstractErfsSurveyScenario()`, on peut jouer sur:
- l'input des variables sur 3 ans
-  ...


In [5]:
log.debug("Create FranceTaxBenefitSystem")

# Instanciation du scenario
pipeline_survey_scenario = PipelineErfsSurveyScenario(
    annee_donnees=annee_donnees,
    period=annee_de_calcul,
    baseline_tax_benefit_system=pipeline_tbs,
    rebuild_input_data=False,
    collection=collection,
    survey_name=survey_name,
)

[leximpact-common-python-libraries DEBUG @ 15:06:53] Create FranceTaxBenefitSystem


LeximpactErfsSurveyScenario : Using /home/cgl/.config/openfisca-survey-manager as config_files_directory
years_available=[2019] vs years=[2021]
Données du scénario : 
 {'input_data_table_by_entity_by_period': {2021: {'individu': 'individu_2019', 'famille': 'famille_2019', 'foyer_fiscal': 'foyer_fiscal_2019', 'menage': 'menage_2019'}}, 'survey': 'leximpact_2019', 'config_files_directory': '/home/cgl/.config/openfisca-survey-manager'}


In [6]:
if rebuild_imputation:
    pipeline_survey_scenario.build_imputation(year=annee_pote)

Injection de revenu_categoriel_foncier


Injection de rente_viagere_titre_onereux_net
Injection de revenus_capitaux_prelevement_bareme
Injection de revenus_capitaux_prelevement_forfaitaire_unique_ir
Injection de revenus_capitaux_prelevement_liberatoire


/home/cgl/leximpact/leximpact-prepare-data/.venv/lib/python3.10/site-packages/openfisca_france/model/prelevements_obligatoires/prelevements_sociaux/cotisations_sociales/travail_non_salarie.py:199: RuntimeWarning: divide by zero encountered in divide
  - (0.007 * (assiette_pss > 5) * ((assiette_pss - 5) / assiette_pss))
/home/cgl/leximpact/leximpact-prepare-data/.venv/lib/python3.10/site-packages/openfisca_france/model/prelevements_obligatoires/prelevements_sociaux/cotisations_sociales/travail_non_salarie.py:199: RuntimeWarning: invalid value encountered in multiply
  - (0.007 * (assiette_pss > 5) * ((assiette_pss - 5) / assiette_pss))


Seed used : 25


 73%|███████▎  | 68/93 [00:10<00:05,  4.29it/s]/home/cgl/leximpact/leximpact-prepare-data/.venv/lib/python3.10/site-packages/openfisca_france/model/prelevements_obligatoires/prelevements_sociaux/cotisations_sociales/travail_non_salarie.py:199: RuntimeWarning: divide by zero encountered in divide
  - (0.007 * (assiette_pss > 5) * ((assiette_pss - 5) / assiette_pss))
/home/cgl/leximpact/leximpact-prepare-data/.venv/lib/python3.10/site-packages/openfisca_france/model/prelevements_obligatoires/prelevements_sociaux/cotisations_sociales/travail_non_salarie.py:199: RuntimeWarning: invalid value encountered in multiply
  - (0.007 * (assiette_pss > 5) * ((assiette_pss - 5) / assiette_pss))
 80%|███████▉  | 74/93 [00:11<00:05,  3.27it/s]

...

 47%|████▋     | 44/94 [00:03<00:05,  8.92it/s]/home/cgl/leximpact/leximpact-prepare-data/.venv/lib/python3.10/site-packages/openfisca_france/model/prelevements_obligatoires/prelevements_sociaux/cotisations_sociales/travail_non_salarie.py:199: RuntimeWarning: divide by zero encountered in divide
  - (0.007 * (assiette_pss > 5) * ((assiette_pss - 5) / assiette_pss))
/home/cgl/leximpact/leximpact-prepare-data/.venv/lib/python3.10/site-packages/openfisca_france/model/prelevements_obligatoires/prelevements_sociaux/cotisations_sociales/travail_non_salarie.py:199: RuntimeWarning: invalid value encountered in multiply
  - (0.007 * (assiette_pss > 5) * ((assiette_pss - 5) / assiette_pss))
/home/cgl/leximpact/leximpact-prepare-data/.venv/lib/python3.10/site-packages/openfisca_france/model/prelevements_obligatoires/prelevements_sociaux/cotisations_sociales/travail_non_salarie.py:199: RuntimeWarning: divide by zero encountered in divide
  - (0.007 * (assiette_pss > 5) * ((assiette_pss - 5) / assi

Seed used : 25


  2%|▏         | 2/94 [00:00<00:24,  3.82it/s]

Seed used : 25


100%|██████████| 94/94 [00:32<00:00,  2.92it/s]


In [7]:
pipeline_survey_scenario.save_current_survey(
    variables=pipeline_survey_scenario.used_as_input_variables,
    collection="leximpact",
    survey_name=f"leximpact_{annee_de_calcul}",
    period=annee_de_calcul,
)

Ajout des cases supplémentaires P et T pour l'impôt sur le revenu à partir de l'ERFS sur le CASD. Fait un peu rapidement, peut être à améliorer.

In [8]:
if rebuild_imputation:
    add_cases_ir_from_erfs(annee_de_calcul)

In [8]:
pipeline_survey_scenario.simulations["baseline"].compute_aggregate("irpp", period=2021)

/home/cgl/leximpact/leximpact-prepare-data/.venv/lib/python3.10/site-packages/openfisca_france/model/prelevements_obligatoires/prelevements_sociaux/cotisations_sociales/travail_non_salarie.py:199: RuntimeWarning: divide by zero encountered in divide
  - (0.007 * (assiette_pss > 5) * ((assiette_pss - 5) / assiette_pss))
/home/cgl/leximpact/leximpact-prepare-data/.venv/lib/python3.10/site-packages/openfisca_france/model/prelevements_obligatoires/prelevements_sociaux/cotisations_sociales/travail_non_salarie.py:199: RuntimeWarning: invalid value encountered in multiply
  - (0.007 * (assiette_pss > 5) * ((assiette_pss - 5) / assiette_pss))


-80804260657.97372